# MiniCPM-2B Parameter Efficient Finetuning（LoRA）A100 80G Single Card Example

显存更小的显卡可用 batch size 和 grad_accum 间时间换空间

## Lowest HW Condition

- VRAM：12GB
- Ampere Architecture
- RAM: 16GB


## 1. 准备数据集

将数据集转换为更通用的格式


In [ ]:
# 转换为 ChatML 格式
import os
import shutil
import json

input_dir = "data/AdvertiseGen"
output_dir = "data/AdvertiseGenChatML"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

for fn in ["train.json", "dev.json"]:
    data_out_list = []
    with open(os.path.join(input_dir, fn), "r") as f, open(
        os.path.join(output_dir, fn), "w"
    ) as fo:
        for line in f:
            if len(line.strip()) > 0:
                data = json.loads(line)
                data_out = {
                    "messages": [
                        {
                            "role": "user",
                            "content": data["content"],
                        },
                        {
                            "role": "assistant",
                            "content": data["summary"],
                        },
                    ]
                }
                data_out_list.append(data_out)
        json.dump(data_out_list, fo, ensure_ascii=False, indent=4)

## 2. 使用 LoRA 进行微调

命令行一键运行


In [ ]:
!bash lora_finetune.sh

## 推理验证


In [ ]:
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
path = "output/AdvertiseGenLoRA/20240315224356/checkpoint-3000"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForCausalLM.from_pretrained(
    path, torch_dtype=torch.bfloat16, device_map="cuda", trust_remote_code=True
)

In [ ]:
res, history = model.chat(
    tokenizer,
    query="<用户>类型#上衣*材质#牛仔布*颜色#白色*风格#简约*图案#刺绣*衣样式#外套*衣款式#破洞<AI>",
    max_length=80,
    top_p=0.5,
)
res, history